In [1]:
import time
import scrapy
from scrapy.crawler import CrawlerProcess
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
items = []

In [3]:
class SmartFitSpider(scrapy.Spider):
    name = 'smart-fit'
    start_urls = ['https://www.smartfit.com.br/unidades']
    
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless') 

    def parse(self, response):
        locations = response.css('div.Locations__item')
        
        for location in locations:
            item = {}
            unit = location.css('h3 > a::text').extract_first()
            item['Unidade'] = unit
            
            plans = location.css('div.Locations__plans__item')
            for plan in plans:
                plan_name = plan.css('h4::text').extract_first()
                integer = plan.css('div.component_location_price__current > div.component_location_price__integer::text').extract_first()
                decimal = plan.css('div.component_location_price__current > div.component_location_price__decimal::text').extract_first()
                current_price = integer + ',' + decimal
                
                item[plan_name] = current_price

            full_address = location.css('div.Locations__content__wrapper > p.Text::text').extract()
            
            address, neighborhood = full_address[0].split('-')
            address, neighborhood = address[:-1], neighborhood[1:]
            
            city_state, code = full_address[-1].split('-')
            city_state, code = city_state[:-1], code[1:]
            
            city, state = city_state.split(',')
            state = state[1:]
            
            item['Endereco'] = address
            item['Bairro'] = neighborhood
            item['Cidade'] = city
            item['Estado'] = state
            item['CEP'] = code
            
            yield item
            
            driver = webdriver.Chrome('../chromedriver.exe', options=self.options)
            driver.get(response.url)
            
            wait=  WebDriverWait(driver, 10)
            print('LETS TRY %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
            load_more = wait.until(ec.presence_of_element_located((By.XPATH, "//button[@class='Button Button--link js-next-page-button']")))
            #load_more = driver.find_element_by_css_selector('body > div.wrap-results-container.js-wrap-results-container > section > div > div > div.Container--center.js-next-page > button')
            print('HALF WAY *************************************')
            load_more.click()
            time.sleep(5)
            print('YEAH! ################################')

            driver.close()

In [4]:
process = CrawlerProcess()
process.crawl(SmartFitSpider)
process.start()

2019-07-21 22:23:07 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-07-21 22:23:07 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Windows-10-10.0.17134-SP0
2019-07-21 22:23:07 [scrapy.crawler] INFO: Overridden settings: {}
2019-07-21 22:23:07 [scrapy.extensions.telnet] INFO: Telnet Password: 90eeb13154e961bc
2019-07-21 22:23:07 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2019-07-21 22:23:07 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defa

LETS TRY %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
HALF WAY *************************************


In [5]:
items

[]